**Author: Átila Bernardo Mota Sousa**

# **Introduction**

This project aims to implement a deep learning model to identify, through fighter statistics and fight history, which are the attributes that most lead a fighter to win.

To achieve this goal, statistics were collected from all active fighters on the UFC website using the web scraping technique. Another dataset of all UFC fights from 1993 to 2021 was also used.

Through the two mentioned datasets, the training dataset of the model was created, containing, for each fight, its result and the statistics of the two fighters.

The created model receives the statistics of the two fighters that will fight and returns a prediction of who will win.

For web scraping, the BeautifulSoup library was used.

To manipulate the neural network, the TensorFlow library was used.


# **Web Scraping**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy
import math
import csv

In [ ]:
# Defining url and headers
url = 'https://www.ufc.com.br/athletes/all'
headers = { 'UserAgent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}

In [ ]:
# Site request
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
print(soup)


<!DOCTYPE html>

<html dir="ltr" lang="pt-br" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"3cad7d7fc7",applicationID:"190390077"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n,r){return function(){return s.recordSupportability("API/"+e+"/called"),o(t+e,[u.now()].concat(c(arguments)),n?null:this,r),n?void 0:this}}var o=t("handle"),a=t(9),c=t(10),f=t("ee").get("tracer"),u=t("loader"),s=t(4),d=NREUM;"undefined"==typeof window.newrelic&&(newrelic=d);var p=["setPageViewName","setCustomAttribute","setErrorHandler","fi

In [ ]:
# Scraping number of fighters
#num_fighters = soup.find('label', class_='option').get_text().strip()
num_fighters = soup.find('div', class_='althelete-total').get_text().strip()
index = num_fighters.find(' ')
num_fighters = int(num_fighters[:index])
print(f'Total of fighters = {num_fighters}')

AttributeError: ignored

In [ ]:
# Scraping name of all fighters
fighters_names = []
all_fighters = []

for i in range(math.ceil(num_fighters/11)):
  url = f'https://www.ufc.com.br/athletes/all?gender=All&search=&page={i}'
  site = requests.get(url, headers=headers)
  soup = BeautifulSoup(site.content, 'html.parser')
  fighters_names.append(soup.find_all('span', class_=re.compile('c-listing-athlete__name')))
  print(f'Total of fighters names scrapeds = {i*11}...')

for i in range(len(fighters_names)):
    all_fighters.extend([a.text.strip() for a in fighters_names[i]])

all_fighters = list(dict.fromkeys(all_fighters))

print(f'{len(all_fighters)} fighters found')
print(all_fighters)

Total fighters names scrapeds = 0...
Total fighters names scrapeds = 11...
Total fighters names scrapeds = 22...
Total fighters names scrapeds = 33...
Total fighters names scrapeds = 44...
Total fighters names scrapeds = 55...
Total fighters names scrapeds = 66...
Total fighters names scrapeds = 77...
Total fighters names scrapeds = 88...
Total fighters names scrapeds = 99...
Total fighters names scrapeds = 110...
Total fighters names scrapeds = 121...
Total fighters names scrapeds = 132...
Total fighters names scrapeds = 143...
Total fighters names scrapeds = 154...
Total fighters names scrapeds = 165...
Total fighters names scrapeds = 176...
Total fighters names scrapeds = 187...
Total fighters names scrapeds = 198...
Total fighters names scrapeds = 209...
Total fighters names scrapeds = 220...
Total fighters names scrapeds = 231...
Total fighters names scrapeds = 242...
Total fighters names scrapeds = 253...
Total fighters names scrapeds = 264...
Total fighters names scrapeds = 275.

In [ ]:
# Scraping statistics of each fighter
fighters_statistics = []
k=-1
for name in all_fighters:
  k+=1
  print(f'Total of fighters scrapeds = {k}')
  print(f'Scraping {name}...')

  name_id = name.lower()
  parts = name_id.split(' ')

  name_id = re.sub(r"[^a-zA-Z0-9]+",'', parts[0])
  for j in range(1, len(parts)):
    if re.sub(r"[^a-zA-Z0-9]+",'', parts[j]) != "":
      name_id += '-'+re.sub(r"[^a-zA-Z0-9]+",'', parts[j])

  url = f'https://www.ufc.com.br/athlete/{name_id}'
  print(url)

  site = requests.get(url, headers=headers)
  soup = BeautifulSoup(site.content, 'html.parser')

  statistic_divs = []
  statistic_divs.append(soup.find_all('div', class_='stats-records-compare.stats-records-inner'))
  statistic_divs.append(soup.find_all('div', class_='stats-records-inner'))

  strike_accuracy = None
  takedown_accuracy = None
  connected_strikes_per_minute = None
  absorved_strikes_per_minute = None
  strikes_defense = None
  takedown_defense = None
  strikes = None
  clinch = None
  ground_and_pound = None
  ko_tko_wins = None
  dec_wins = None 
  fin_wins = None

  for divs in statistic_divs:
    for div in divs:
      label_elements = div.find_all('h2', class_=re.compile('e-t3'))

      for label_element in label_elements:
        label = label_element.get_text().strip()
        #match label:
        if label == "Precisão de striking":
          try:
            strike_accuracy = div.find('text', class_=re.compile('e-chart-circle__percent')).get_text().strip()
          except:
            pass
        if label == "Precisão De Quedas":
          try:
            takedown_accuracy = div.find('text', class_=re.compile('e-chart-circle__percent')).get_text().strip()
          except:
            pass

      label_elements = soup.find_all('div', class_='c-stat-compare__label')

  
      for label_element in label_elements:
        parent = label_element.parent
        label = label_element.get_text().strip()
        #match label:
        if label == "Golpes Sig. Conectados":
          try:
            connected_strikes_per_minute = parent.find('div', class_='c-stat-compare__number').get_text().strip()
          except:
            pass
        if label == "Golpes Sig. Absorvidos":
          try:
            absorved_strikes_per_minute = parent.find('div', class_='c-stat-compare__number').get_text().strip()
          except:
            pass
        if label == "Defesa de Golpes Sig.":
          try:
            strikes_defense = parent.find('div', class_='c-stat-compare__number').get_text().strip()
          except:
            pass
        if label == "Defesa De Quedas":
          try:
            takedown_defense = parent.find('div', class_='c-stat-compare__number').get_text().strip()
          except:
            pass


        label_elements = label_element = soup.find_all('h2', class_=re.compile('c-stat-3bar__title'))

        for label_element in label_elements:
          parent = label_element.parent
          label = label_element.get_text().strip()
          #match label:
          if label == "Golpes Sig. Por Posição":
            statistics = parent.find_all('div', class_=re.compile('c-stat-3bar__value'))
            i = 0
            for statistic in statistics: 
            # match i:
                if i == 0:
                  strikes = statistic.get_text().strip()
                if i == 1:
                  clinch = statistic.get_text().strip()
                if i == 2:
                  ground_and_pound = statistic.get_text().strip()
                
                i+=1


          if label == "Win by Method":
              statistics = parent.find_all('div', class_=re.compile('c-stat-3bar__value'))
              i = 0
              for statistic in statistics: 
                #match i:
                  if i==0:
                    ko_tko_wins = statistic.get_text().strip()
                  if i==1:
                    dec_wins = statistic.get_text().strip()
                  if i==2:
                    fin_wins = statistic.get_text().strip()

                  i+=1

  # Data Cleaning

  if strikes_defense:
    parts = strikes_defense.split('\n')
    strikes_defense = parts[0] + '%'

  if takedown_defense:
    parts = takedown_defense.split('\n')
    takedown_defense = parts[0] + '%'

  if strikes:
    index1 = strikes.find('(')
    index2 = strikes.find(')')
    strikes = strikes[index1+1:index2]

  if clinch:
    index1 = clinch.find('(')
    index2 = clinch.find(')')
    clinch = clinch[index1+1:index2]

  if ground_and_pound:
    index1 = ground_and_pound.find('(')
    index2 = ground_and_pound.find(')')
    ground_and_pound = ground_and_pound[index1+1:index2]

  if ko_tko_wins:
    index1 = ko_tko_wins.find('(')
    index2 = ko_tko_wins.find(')')
    ko_tko_wins = ko_tko_wins[index1+1:index2]

  if dec_wins:
    index1 = dec_wins.find('(')
    index2 = dec_wins.find(')')
    dec_wins = dec_wins[index1+1:index2]

  if fin_wins:
    index1 = fin_wins.find('(')
    index2 = fin_wins.find(')')
    fin_wins = fin_wins[index1+1:index2]


  # Writing fighters statistics on .csv file

  fighters_statistics.append([name, strike_accuracy, takedown_accuracy, connected_strikes_per_minute, absorved_strikes_per_minute, strikes_defense, takedown_defense, strikes,
          clinch, ground_and_pound, ko_tko_wins, dec_wins, fin_wins])

#print("Precisão de Strikes: " + strike_accuracy)
#print("Precisão de Quedas: " + takedown_accuracy)
#print("Golpes Sig. Conectados: " + connected_strikes_per_minute)
#print("Golpes Sig. Absorvidos: " + absorved_strikes_per_minute) 
#print("Defesa de Golpes Sig.: " + strikes_defense) 
#print("Defesa de Quedas: " + takedown_defense) 
#print("Golpes Sig. Por Posição - Em pé: " + strikes)
#print("Golpes Sig. Por Posição - Clinch: " + clinch)
#print("Golpes Sig. Por Posição - Solo: " + ground_and_pound)
#print("Win By Method - KO/TKO: " + ko_tko_wins)
#print("Win By Method - Dec: " + dec_wins)
#print("Win By Method - Fin: " + fin_wins)

# Creating .csv file

head = ['name', 'strike_accuracy', 'takedown_accuracy', 'connected_strikes_per_minute', 'absorved_strikes_per_minute', 'strikes_defense', 'takedown_defense', 'strikes',
        'clinch', 'ground_and_pound', 'ko_tko_wins', 'dec_wins', 'fin_wins']

with open('ufc_all_fighters_statistics.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(head)

    # write multiple rows
    writer.writerows(fighters_statistics)


A saída de streaming foi truncada nas últimas 5000 linhas.
Scraping Max Holloway...
https://www.ufc.com.br/athlete/max-holloway
Total of fighters scrapeds = 1052
Scraping Holly Holm...
https://www.ufc.com.br/athlete/holly-holm
Total of fighters scrapeds = 1053
Scraping Rex Holman...
https://www.ufc.com.br/athlete/rex-holman
Total of fighters scrapeds = 1054
Scraping Joseph Holmes...
https://www.ufc.com.br/athlete/joseph-holmes
Total of fighters scrapeds = 1055
Scraping Kurt Holobaugh...
https://www.ufc.com.br/athlete/kurt-holobaugh
Total of fighters scrapeds = 1056
Scraping Paddy Holohan...
https://www.ufc.com.br/athlete/paddy-holohan
Total of fighters scrapeds = 1057
Scraping Mark Holst...
https://www.ufc.com.br/athlete/mark-holst
Total of fighters scrapeds = 1058
Scraping Scott Holtzman...
https://www.ufc.com.br/athlete/scott-holtzman
Total of fighters scrapeds = 1059
Scraping Sabah Homasi...
https://www.ufc.com.br/athlete/sabah-homasi
Total of fighters scrapeds = 1060
Scraping Mark 

In [ ]:
data = pd.read_csv('./ufc_all_fighters_statistics.csv')

In [ ]:
# Check for null values
data.isnull().sum()

name                              0
strike_accuracy                 452
takedown_accuracy               722
connected_strikes_per_minute    445
absorved_strikes_per_minute     445
strikes_defense                 457
takedown_defense                858
dtype: int64

In [ ]:
# drop dirty columns
data.drop('ko_tko_wins', inplace=True, axis=1)
data.drop('dec_wins', inplace=True, axis=1)
data.drop('fin_wins', inplace=True, axis=1)

data.drop('strikes', inplace=True, axis=1)
data.drop('clinch', inplace=True, axis=1)
data.drop('ground_and_pound', inplace=True, axis=1)

In [ ]:
data.head()

,name,strike_accuracy,takedown_accuracy,connected_strikes_per_minute,absorved_strikes_per_minute,strikes_defense,takedown_defense
0,Asjabharan _,NaN,NaN,NaN,NaN,NaN,NaN
1,Angga -,39%,0%,3.56,3.77,53%,33%
2,Danny Abbadi,38%,NaN,3.29,4.41,58%,78%
3,Nariman Abbassov,20%,0%,3.00,5.67,46%,67%
4,Tank Abbott,39%,NaN,2.41,10.03,38%,67%


In [ ]:
# dropna on .csv

data = data.dropna()
data.to_csv('ufc_all_fighters_statistics_dropna.csv')

# **Deep Learning**

##**0. Import Data**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#reading ufc fighters statistics

ufc_fighters_statistics = pd.read_csv("ufc_all_fighters_statistics_dropna.csv")
ufc_fighters_statistics.head()

,Unnamed: 0,name,strike_accuracy,takedown_accuracy,connected_strikes_per_minute,absorved_strikes_per_minute,strikes_defense,takedown_defense,strikes,clinch,ground_and_pound
0,1,Angga -,39%,0%,3.56,3.77,53%,33%,88%,12%,0%
1,3,Nariman Abbassov,20%,0%,3.00,5.67,46%,67%,100%,0%,0%
2,6,Shamil Abdurakhimov,44%,23%,2.60,2.80,58%,47%,67%,24%,9%
3,8,Papy Abedi,55%,57%,2.80,3.15,49%,50%,37%,35%,28%
4,9,Klidson Abreu,41%,20%,2.05,2.90,56%,80%,90%,10%,0%


In [ ]:
#reading ufc fights results from 1993 to 2021

ufc_fights_results = pd.read_csv("data.csv")
ufc_fights_results.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,B_avg_opp_SIG_STR_pct,B_avg_TD_pct,B_avg_opp_TD_pct,B_avg_SUB_ATT,B_avg_opp_SUB_ATT,B_avg_REV,B_avg_opp_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_avg_TD_att,B_avg_TD_landed,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_BODY_att,B_avg_BODY_landed,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_LEG_att,B_avg_LEG_landed,...,R_avg_opp_BODY_landed,R_avg_LEG_att,R_avg_LEG_landed,R_avg_opp_LEG_att,R_avg_opp_LEG_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_opp_DISTANCE_att,R_avg_opp_DISTANCE_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_opp_CLINCH_att,R_avg_opp_CLINCH_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_opp_GROUND_att,R_avg_opp_GROUND_landed,R_avg_CTRL_time(seconds),R_avg_opp_CTRL_time(seconds),R_total_time_fought(seconds),R_total_rounds_fought,R_total_title_bouts,R_current_win_streak,R_current_lose_streak,R_longest_win_streak,R_wins,R_losses,R_draw,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,0.420000,0.49500,0.330,0.36000,0.500,1.0000,0.00,0.0,50.0000,20.0000,84.000,45.0000,76.5000,41.000,114.000,64.00,1.500,1.0,9.0,6.500,39.500,11.000,63.0000,27.500,7.5000,7.0000,12.0000,9.0000,3.000,2.000,...,1.000000,0.000000,0.000000,0.000000,0.000000,33.000000,16.000000,12.000000,6.000000,1.000000,1.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,54.000000,166.000000,1,0,1,0,1,1,0,0,0,0,0,1,0,0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,0.660000,0.30500,0.300,0.50000,1.500,0.0000,0.00,0.0,65.5000,35.0000,50.000,16.5000,113.5000,68.500,68.500,29.00,2.500,1.5,0.5,0.500,46.000,20.000,36.0000,7.500,12.0000,8.0000,8.0000,3.0000,7.500,7.000,...,6.156250,1.781250,1.781250,9.656250,8.093750,58.312500,30.437500,63.312500,23.125000,6.968750,3.656250,5.0000,4.125000,12.281250,9.062500,0.843750,0.343750,176.156250,154.687500,747.343750,17,0,2,0,2,4,2,0,0,1,0,3,0,0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.718750,10.953125,9.046875,3.390625,3.140625,45.875000,20.281250,36.343750,18.687500,11.328125,8.562500,1.4375,1.171875,2.031250,1.531250,6.031250,4.937500,82.453125,89.781250,448.203125,13,0,3,0,3,4,3,0,0,0,1,3,0,0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Switch,160.02,160.02,115.0,28.0,25.0
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,0.535625,0.57875,0.185,0.16625,0.125,0.1875,0.25,0.0,109.1875,57.9375,50.625,28.4375,170.4375,105.625,74.375,44.25,5.375,1.5,2.0,0.625,77.375,31.375,41.5625,22.625,31.1875,26.1875,7.6875,4.9375,0.625,0.375,...,31.658203,5.683594,4.862305,16.903320,14.203125,85.845703,27.907227,106.913086,48.479492,18.996094,13.970703,35.3125,32.059570,4.489258,2.484375,6.621094,4.015625,141.285156,310.152344,874.380859,30,0,2,0,2,5,6,0,0,0,1,2,2,0,Orthodox,167.64,172.72,1

In [ ]:
print(len(ufc_fights_results))

6012


In [ ]:
# Creating train data

# Creating .csv train file

#head = ['winner', 'r_strike_accuracy', 'r_takedown_accuracy', 'r_connected_strikes_per_minute', 'r_absorved_strikes_per_minute', 'r_strikes_defense', 'r_takedown_defense', 'r_strikes',
        #'r_clinch', 'r_ground_and_pound', 'r_ko_tko_wins', 'r_dec_wins', 'r_fin_wins','b_strike_accuracy', 'b_takedown_accuracy', 'b_connected_strikes_per_minute',
        #'b_absorved_strikes_per_minute', 'b_strikes_defense','b_takedown_defense', 'b_strikes','b_clinch', 'b_ground_and_pound', 'b_ko_tko_wins', 'b_dec_wins', 'b_fin_wins']

#head = ['winner', 'r_strike_accuracy', 'r_takedown_accuracy', 'r_connected_strikes_per_minute', 'r_absorved_strikes_per_minute', 'r_strikes_defense', 'r_takedown_defense', 'r_strikes',
        #'r_clinch', 'r_ground_and_pound','b_strike_accuracy', 'b_takedown_accuracy', 'b_connected_strikes_per_minute','b_absorved_strikes_per_minute', 'b_strikes_defense',
        #'b_takedown_defense', 'b_strikes','b_clinch', 'b_ground_and_pound']

head = ['winner', 'r_strike_accuracy', 'r_takedown_accuracy', 'r_connected_strikes_per_minute', 'r_absorved_strikes_per_minute', 'r_strikes_defense', 'r_takedown_defense',
        'b_strike_accuracy', 'b_takedown_accuracy', 'b_connected_strikes_per_minute','b_absorved_strikes_per_minute', 'b_strikes_defense','b_takedown_defense']

with open('train_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(head)

    # write multiple rows
    for index, row in ufc_fights_results.iterrows():
      if row['Winner'] == "Draw":
        continue
        
      row_values = []
      #print("Red Fighter: " + row['R_fighter'])
      #print("Blue Fighter: " + row['B_fighter'])
      #print("Winner: " + row['Winner'])

      row_values.extend([str(row['Winner'])])

      r_fighter_statistics = ufc_fighters_statistics.loc[ufc_fighters_statistics['name'] == row['R_fighter']]
      #print(r_fighter_statistics)
      if r_fighter_statistics.empty:
        print(f'Não achou estatísticas de {row["R_fighter"]}')
      else:
        print(f'Achou estatísticas de {row["R_fighter"]}')


      b_fighter_statistics = ufc_fighters_statistics.loc[ufc_fighters_statistics['name'] == row['B_fighter']]
      if b_fighter_statistics.empty:
        print(f'Não achou estatísticas de {row["B_fighter"]}')
      else:
        print(f'Achou estatísticas de {row["B_fighter"]}')

      #print("Red Fighter Statistics: ")
      for column in r_fighter_statistics:
        if column != "name" and column != "Unnamed: 0":
          #print(column)
          value = r_fighter_statistics.get([column]).values.T

          if value[0] != 0:
            stat = str(value[0][0])
          else:
            continue

        
          if stat.find('%') != -1:
            parts = stat.split('%')
            stat_int = int(parts[0].strip())/100
            stat = str(stat_int)

          #print(stat)
          row_values.extend([float(stat)])


      #print("Blue Fighter Statistics: ")
      for column in b_fighter_statistics:
        if column != "name" and column != "Unnamed: 0":
          #print(column)
          value = b_fighter_statistics.get([column]).values.T

          if value[0] != 0:
            stat = str(value[0][0])
          else:
            continue

        
          if stat.find('%') != -1:
            parts = stat.split('%')
            stat_int = int(parts[0].strip())/100
            stat = str(stat_int)

          #print(stat)
          row_values.extend([float(stat)])

      if len(row_values) == 19:
        #print("Wrinting row: ")
        #print(row_values)
        writer.writerow(row_values)


Não achou estatísticas de Adrian Yanez
Achou estatísticas de Gustavo Lopez
Achou estatísticas de Trevin Giles
Não achou estatísticas de Roman Dolidze
Achou estatísticas de Tai Tuivasa
Não achou estatísticas de Harry Hunsucker
Não achou estatísticas de Cheyanne Buys
Não achou estatísticas de Montserrat Conejo
Achou estatísticas de Marion Reneau
Achou estatísticas de Macy Chiasson
Achou estatísticas de Leonardo Santos
Achou estatísticas de Grant Dawson
Não achou estatísticas de Song Kenan
Achou estatísticas de Max Griffin
Achou estatísticas de Derek Brunson
Achou estatísticas de Kevin Holland
Achou estatísticas de Montel Jackson
Achou estatísticas de Jesse Strader
Achou estatísticas de Bruno Silva
Achou estatísticas de JP Buys
Achou estatísticas de Manel Kape
Achou estatísticas de Matheus Nicolau
Achou estatísticas de Misha Cirkunov
Não achou estatísticas de Ryan Spann
Achou estatísticas de Charles Jourdain
Achou estatísticas de Marcelo Rojo
Achou estatísticas de Angela Hill
Achou estatí

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


A saída de streaming foi truncada nas últimas 5000 linhas.
Achou estatísticas de Ramsey Nijem
Achou estatísticas de Justin Edwards
Achou estatísticas de Cole Miller
Achou estatísticas de Sam Sicilia
Achou estatísticas de Lorenz Larkin
Achou estatísticas de Brad Tavares
Achou estatísticas de Vinc Pichel
Achou estatísticas de Garett Whiteley
Achou estatísticas de Alptekin Ozkilic
Achou estatísticas de Louis Smolka
Achou estatísticas de Trevor Smith
Achou estatísticas de Brian Houston
Achou estatísticas de Charlie Brenneman
Achou estatísticas de Beneil Dariush
Achou estatísticas de Mairbek Taisumov
Achou estatísticas de Tae Hyun Bang
Achou estatísticas de Kiichi Kunimoto
Achou estatísticas de Luiz Dutra
Achou estatísticas de Katsunori Kikuno
Achou estatísticas de Quinn Mulhern
Não achou estatísticas de Royston Wee
Não achou estatísticas de Dave Galera
Achou estatísticas de Tarec Saffiedine
Achou estatísticas de Hyun Gyu Lim
Achou estatísticas de Tatsuya Kawajiri
Achou estatísticas de Sean

In [ ]:
df = pd.read_csv("train_data.csv")

In [ ]:
# Data clean

df.drop('r_strikes', inplace=True, axis=1)
df.drop('r_clinch', inplace=True, axis=1)
df.drop('r_ground_and_pound', inplace=True, axis=1)

df.drop('b_strikes', inplace=True, axis=1)
df.drop('b_clinch', inplace=True, axis=1)
df.drop('b_ground_and_pound', inplace=True, axis=1)

In [ ]:
# Data standardization

df['r_strike_accuracy'] = (df['r_strike_accuracy'] - df['r_strike_accuracy'].mean()) / df['r_strike_accuracy'].std() 
df['r_takedown_accuracy'] = (df['r_takedown_accuracy'] - df['r_takedown_accuracy'].mean()) / df['r_takedown_accuracy'].std() 
df['r_connected_strikes_per_minute'] = (df['r_connected_strikes_per_minute'] - df['r_connected_strikes_per_minute'].mean()) / df['r_connected_strikes_per_minute'].std() 
df['r_absorved_strikes_per_minute'] = (df['r_absorved_strikes_per_minute'] - df['r_absorved_strikes_per_minute'].mean()) / df['r_absorved_strikes_per_minute'].std()
df['r_strikes_defense'] = (df['r_strikes_defense'] - df['r_strikes_defense'].mean()) / df['r_strikes_defense'].std() 
df['r_takedown_defense'] = (df['r_takedown_defense'] - df['r_takedown_defense'].mean()) / df['r_takedown_defense'].std() 

df['b_strike_accuracy'] = (df['b_strike_accuracy'] - df['b_strike_accuracy'].mean()) / df['b_strike_accuracy'].std() 
df['b_takedown_accuracy'] = (df['b_takedown_accuracy'] - df['b_takedown_accuracy'].mean()) / df['b_takedown_accuracy'].std() 
df['b_connected_strikes_per_minute'] = (df['b_connected_strikes_per_minute'] - df['b_connected_strikes_per_minute'].mean()) / df['b_connected_strikes_per_minute'].std() 
df['b_absorved_strikes_per_minute'] = (df['b_absorved_strikes_per_minute'] - df['b_absorved_strikes_per_minute'].mean()) / df['b_absorved_strikes_per_minute'].std()
df['b_strikes_defense'] = (df['b_strikes_defense'] - df['b_strikes_defense'].mean()) / df['b_strikes_defense'].std() 
df['b_takedown_defense'] = (df['b_takedown_defense'] - df['b_takedown_defense'].mean()) / df['b_takedown_defense'].std() 



In [ ]:
df.head()

,winner,r_strike_accuracy,r_takedown_accuracy,r_connected_strikes_per_minute,r_absorved_strikes_per_minute,r_strikes_defense,r_takedown_defense,b_strike_accuracy,b_takedown_accuracy,b_connected_strikes_per_minute,b_absorved_strikes_per_minute,b_strikes_defense,b_takedown_defense
0,Blue,-0.352733,1.375152,-0.072460,0.270825,0.488219,-0.749777,0.133760,-0.272534,0.378284,-0.570400,-1.113611,0.165490
1,Blue,0.343670,-0.870548,-0.408643,0.169163,-0.386649,1.436747,0.767039,-0.221131,-0.011669,-0.607186,-1.382203,-1.156236
2,Red,0.482951,-0.456866,0.192947,-0.305257,-0.678272,1.554937,1.273661,0.190092,0.552519,-0.739613,0.095055,-0.550445
3,Red,1.597196,2.202515,0.122172,-1.576024,0.050785,0.313937,1.020350,0.909733,1.531549,1.393934,-1.113611,2.148079
4,Red,0.204389,-0.575061,-0.267092,0.025143,0.196596,0.077556,0.133760,-0.632355,-1.073668,-0.268761,-2.053684,-1.982314


In [ ]:
X = pd.get_dummies(df.drop(['winner'], axis=1))
y = df['winner'].apply(lambda x: 1 if x=="Red" else 0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
X_train.head()

,r_strike_accuracy,r_takedown_accuracy,r_connected_strikes_per_minute,r_absorved_strikes_per_minute,r_strikes_defense,r_takedown_defense,b_strike_accuracy,b_takedown_accuracy,b_connected_strikes_per_minute,b_absorved_strikes_per_minute,b_strikes_defense,b_takedown_defense
2188,-0.213453,0.370497,-0.222858,-0.051103,-0.678272,0.136651,1.147005,1.320957,-0.492887,-1.335535,0.900833,0.826353
515,-0.909856,-0.811451,1.289964,-0.101934,1.946334,1.850413,0.640383,0.447107,2.552064,2.144354,-0.979315,0.055347
2249,-0.213453,1.020568,-0.028226,1.058700,-0.386649,0.845794,-0.119551,0.138689,1.058628,0.334518,0.363648,-0.054797
3297,-0.352733,0.015913,-0.054767,0.067502,0.779842,0.904889,-1.259451,0.498510,-1.654449,-1.181036,-0.307833,-0.440301
812,0.622232,-0.102282,-0.045920,0.550393,-0.386649,0.077556,2.033595,1.834986,2.751189,2.203211,-1.113611,0.991569


In [ ]:
y_train.head()

2188    1
515     1
2249    0
3297    1
812     0
Name: winner, dtype: int64

## **1. Import Dependencies**

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from keras import callbacks
from sklearn.metrics import accuracy_score

## **2. Build and Compile Model**

In [ ]:
model = Sequential()
model.add(Dense(units=13, activation="relu", input_dim=len(X_train.columns)))
#model.add(Dense(units=32, activation="relu"))
#model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=1, activation="tanh"))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

## **3. Fit, Predict and Evaluate**

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True, verbose=1)

model.fit(X, y, epochs=1500, batch_size=128, callbacks =[earlystopping])

Epoch 1/1500
32/32 [==============================] - 0s 1ms/step - loss: 1.0225 - accuracy: 0.6026
Epoch 2/1500
32/32 [==============================] - 0s 1ms/step - loss: 0.7182 - accuracy: 0.6308
Epoch 3/1500
32/32 [==============================] - 0s 1ms/step - loss: 0.6796 - accuracy: 0.6385
Epoch 4/1500
32/32 [==============================] - 0s 998us/step - loss: 0.6569 - accuracy: 0.6400
Epoch 5/1500
32/32 [==============================] - 0s 1ms/step - loss: 0.6431 - accuracy: 0.6482
Epoch 6/1500
32/32 [==============================] - 0s 986us/step - loss: 0.6337 - accuracy: 0.6520
Epoch 7/1500
32/32 [==============================] - 0s 948us/step - loss: 0.6266 - accuracy: 0.6577
Epoch 8/1500
32/32 [==============================] - 0s 1ms/step - loss: 0.6210 - accuracy: 0.6614
Epoch 9/1500
32/32 [==============================] - 0s 1ms/step - loss: 0.6166 - accuracy: 0.6637
Epoch 10/1500
32/32 [==============================] - 0s 984us/step - loss: 0.6128 - accuracy

In [ ]:
y_hat = model.predict(X_test)
y_hat = [0 if val<0.5 else 1 for val in y_hat]

In [ ]:
accuracy_score(y_test,y_hat)

0.7110834371108343

## **4. Saving and Reloading**

In [ ]:
model.save('betUFCmodel')

In [ ]:
!zip -r /content/model.zip /content/betUFCmodel

updating: content/betUFCmodel/ (stored 0%)
updating: content/betUFCmodel/saved_model.pb (deflated 86%)
updating: content/betUFCmodel/variables/ (stored 0%)
updating: content/betUFCmodel/variables/variables.index (deflated 52%)
updating: content/betUFCmodel/variables/variables.data-00000-of-00001 (deflated 58%)
updating: content/betUFCmodel/keras_metadata.pb (deflated 84%)
updating: content/betUFCmodel/assets/ (stored 0%)


In [ ]:
from google.colab import files
files.download("/content/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Results and Discussions**

The highest accuracy achieved until now by the model was approximately 71%.